In [1]:
%pip install fcx-playground

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 29.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.7 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.9/206.9 kB 2.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 6.5 MB/s eta 0:00:0000:01
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 32.2 MB/s eta 0:00:00:00:010:01m
  Created wheel for asciitree: filename=asciitree-0.3.3-py3-none-any.whl size=5032 sha256=5160e865d903375332394d6399b3b6c7d85f1f281770f45545e07b2fc0064b9a
  Stored in directory: /home/sanjog/.cache/pip/wheels/71/c1/da/23077eb3b87d24d6f3852ed1ed1a1ac2d3c885ad6ebd2b4a07
Successfully built asciitree
Note: you may need to restart the kernel to use updated packages.


In [26]:
import os
import shutil
import boto3
import numpy as np
import pandas as pd
import xarray as xr
import zarr

from typing import Generator

from fcx_playground.fcx_dataprocess.tiles_rad_range import RadRangeTilesPointCloudDataProcess

In [46]:
class HIWRAPTilesPointCloudDataProcess(RadRangeTilesPointCloudDataProcess):
    def _cleaning(self, data: xr.Dataset) -> xr.Dataset:
      # data extraction
      # scrape necessary data columns 
      extracted_data = data[['time', 'ref', 'lat', 'lon', 'alt', 'roll', 'pitch', 'head', 'range']]
      return extracted_data

    def _transformation(self, data: xr.Dataset) -> pd.DataFrame:
      #  transform the data to a suitable data formatting
      secs = data['time'].values # seconds since 2013-09-22T00:00:00Z
      lat = data['lat'].values
      lon = data['lon'].values
      alt = data['alt'].values # altitude of aircraft in meters
      roll = data["roll"].values
      pitch = data["pitch"].values
      head = data["head"].values
      ref = data['ref'].values #CRS radar reflectivity #2d data
      rad_range = data["range"].values # has lower count than ref
      
      # time correction and conversion:
      base_time = self._get_date_from_url("2013-09-22")
      delta = secs.astype('timedelta64[s]') + base_time
      time = (delta).astype('timedelta64[s]').astype(np.int64)

      # transform ref to 1d array and repeat other columns to match data dimension

      num_col = ref.shape[0] # number of cols
      num_row = ref.shape[1] # number of rows

      time = np.repeat(time, num_row)
      lon = np.repeat(lon, num_row)
      lat = np.repeat(lat, num_row)
      alt = np.repeat(alt, num_row)
      roll = np.repeat(roll * self.to_rad, num_row)
      pitch = np.repeat(pitch * self.to_rad, num_row)
      head = np.repeat(head * self.to_rad, num_row)
      rad_range = np.tile(rad_range, num_col)
      ref = ref.flatten()

      # curtain creation

      x, y, z = self._down_vector(roll, pitch, head)
      x = np.multiply(x, np.divide(rad_range, 111000 * np.cos(lat * self.to_rad)))
      y = np.multiply(y, np.divide(rad_range, 111000))
      z = np.multiply(z, rad_range)
      lon = np.add(-x, lon)
      lat = np.add(-y, lat)
      alt = np.add(z, alt)

      # sort by time

      sort_idx = np.argsort(time)
      lon = lon[sort_idx]
      lat = lat[sort_idx]
      alt = alt[sort_idx]
      ref = ref[sort_idx]
      time = time[sort_idx]

      # remove nan and infinite using mask (dont use masks filtering for values used for curtain creation)

      mask = np.logical_and(np.isfinite(ref), alt > 0)
      time = time[mask]
      ref = ref[mask]
      lon = lon[mask]
      lat = lat[mask]
      alt = alt[mask]


      df = pd.DataFrame(data = {
        'time': time[::20],
        'lon': lon[::20],
        'lat': lat[::20],
        'alt': alt[::20],
        'ref': ref[::20]
      })
      print(df)
      return df

    def _get_date_from_url(self, url: str) -> np.datetime64:
      # get date from url
      # date is in the format of YYYYMMDD
      # eg. 20190801
      date = url.split("-")
      np_date = np.datetime64('{}-{}-{}'.format("2013", "09", "22"))
      return np_date

In [47]:
obj = HIWRAPTilesPointCloudDataProcess()

In [48]:
# %pip install netCDF4

In [49]:
data = obj.ingest("../../../../tmp/HS3_HIWRAP_20130925_kainnerchirp_175905-183056_v03.nc")

In [50]:
data

<xarray.Dataset>
Dimensions:       (time: 115064, range: 353)
Coordinates:
  * range         (range) float32 74.95 149.9 224.8 ... 2.638e+04 2.646e+04
  * time          (time) float32 3.239e+05 3.239e+05 ... 3.259e+05 3.259e+05
Data variables: (12/34)
    alt           (time) float32 ...
    dopcorr       (time, range) float32 ...
    doph          (time, range) float32 ...
    dophu         (time, range) float32 ...
    dopl          (time, range) float32 ...
    doplh         (time, range) float32 ...
    ...            ...
    vacft         (time) float32 ...
    vuh           float32 ...
    vul           float32 ...
    vulh          float32 ...
    wvel          (time) float32 ...
    year          int16 ...
Attributes:
    title:        HIWRAP Data, NASA Goddard Space Flight Center
    filename:     /karldata4/tian/HS3/netcdf/2013/dist/dat/v3/hs3_hiwrap_kain...
    experiment:   NASA HS3
    source:       created from HIWRAP binary file, Version 0.0
    comments:     Contact: lin.tian-1@nasa.gov
    Conventions:  CF-1.6
    history:      Wed Sep 23 15:20:50 2015: ncks -4 -L 4 -O /ghrc/ops/output/...
    NCO:          4.3.7

In [51]:
pre_processed_data = obj.preprocess(data)

           time        lon        lat           alt        ref
0        323945 -73.095528  29.753232  16659.519531  21.879847
1        323945 -73.091836  29.754161   5575.201172  11.916979
2        323945 -73.092493  29.754017   7297.770508   4.690439
3        323945 -73.090665  29.754418   2504.533203  30.471460
4        323945 -73.090780  29.754393   2804.111328  26.959787
...         ...        ...        ...           ...        ...
1302770  325899 -72.360062  28.536817  10738.058594 -10.496276
1302771  325899 -72.361389  28.536723   7367.893555  -1.636184
1302772  325899 -72.360504  28.536786   9614.669922  -7.572555
1302773  325899 -72.358175  28.536950  15531.183594  11.820386
1302774  325899 -72.357792  28.536977  16504.787109  12.811761

[1302775 rows x 5 columns]


In [52]:
point_clouds_tileset = obj.prep_visualization(pre_processed_data)

In [53]:
point_clouds_tileset

'temp/2013-09-22/zarr_point_cloud'